### Big Query

In [95]:
import logging
import os
import re
from typing import Optional

import pandas as pd
from google.cloud import bigquery
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/bigquery"
]

# Google Sheet IDs
SPREADSHEET_ID = '1gq3blPuyAGbPEqc-VjW78S-J5lAJpDFAvWZo1j4ZuYY'
SHEET_NAME = 'Results'
PROJECT_ID = 'pccw-internal-virtual-agent'
DATASET_ID = 'prod_wanda_ds'
TABLE_ID = 'dialogflow_bigquery_export_data'

class TextClusteringPipeline:
    def __init__(self, spreadsheet_id: str, sheet_name: str, project_id: str, dataset_id: str, table_id: str):
        self.spreadsheet_id = spreadsheet_id
        self.sheet_name = sheet_name
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.table_id = table_id
        self.creds = self.authenticate_google_api()
        self.service = build('sheets', 'v4', credentials=self.creds)
        self.sheet = self.service.spreadsheets()
        self.client = bigquery.Client(credentials=self.creds, project=self.project_id)
        self.model = SentenceTransformer('all-MiniLM-L12-v2')

    def authenticate_google_api(self) -> Optional[Credentials]:
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                try:
                    creds.refresh(Request())
                except Exception as e:
                    logging.error(f"Error refreshing credentials: {e}")
                    return None
            else:
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
                with open('token.json', 'w') as token:
                    token.write(creds.to_json())
        return creds

    def get_data_from_bigquery(self, query: str) -> pd.DataFrame:
        try:
            job = self.client.query(query)
            result = job.result().to_dataframe()
            logging.info("Data successfully retrieved from BigQuery.")
            return result
        except Exception as e:
            logging.error(f"Failed to retrieve data from BigQuery: {e}")
            return pd.DataFrame()

    def update_google_sheet(self, data: pd.DataFrame):
        if data.empty:
            logging.error("Received empty DataFrame, aborting Google Sheet update.")
            return
        try:
            header_values = [["Cluster", "Question", "Sentence Count", "Topic"]]
            self.sheet.values().update(
                spreadsheetId=self.spreadsheet_id, range=f'{self.sheet_name}!A1',
                valueInputOption='RAW', body={'values': header_values}
            ).execute()
            logging.info("Header updated in Google Sheet.")

            update_values = data[["Cluster", "Question", "Sentence Count", "Topic"]].values.tolist()
            result = self.sheet.values().update(
                spreadsheetId=self.spreadsheet_id, range=f'{self.sheet_name}!A2',
                valueInputOption='RAW', body={'values': update_values}
            ).execute()
            logging.info(f"{result.get('updatedCells')} cells updated in Google Sheet.")
        except Exception as e:
            logging.error(f"Failed to update Google Sheet: {e}")

    @staticmethod
    def clean_text(text: str) -> str:
        patterns = [
            (r'<@U\w+>', ''),
            (r'[^\w]ticket[\w]id[\w]redacted', 'Ticket Number'),
            (r'INC00\w+', 'Ticket Number'),
            (r'(?i)console connect', ''),
            (r'(?i)wanda', 'Chatbot'),
            (r'(?i)SR\d+', 'Circuit Number')
        ]
        for pattern, replacement in patterns:
            text = re.sub(pattern, replacement, text)
        return text

    def run(self):
        logging.info("Pipeline started.")
        query = """
                SELECT JSON_VALUE(request, '$.queryInput.text.text') AS Request
                FROM `pccw-internal-virtual-agent.prod_wanda_ds.dialogflow_bigquery_export_data`
                ORDER BY conversation_name ASC, request_time ASC
                """
        data = self.get_data_from_bigquery(query)
        if data.empty:
            logging.error("No data retrieved, aborting pipeline.")
            return

        clean_text = [self.clean_text(text) for text in data['Request'].tolist()]

        corpus_embeddings = self.model.encode(clean_text)
        logging.info("Embeddings generated.")
        
        # Perform agglomerative clustering
        clustering_model = AgglomerativeClustering(
            n_clusters=None, distance_threshold=1.5
        )  # , affinity='cosine', linkage='average', distance_threshold=0.4)
        clustering_model.fit(corpus_embeddings)
        cluster_assignment = clustering_model.labels_
        
        clustered_sentences = {}
        for sentence_id, cluster_id in enumerate(cluster_assignment):
            if cluster_id not in clustered_sentences:
                clustered_sentences[cluster_id] = []
        
            clustered_sentences[cluster_id].append(clean_text[sentence_id])

        # Find the key sentence for each cluster
        key_sentences = {}
        for cluster_id, sentences in clustered_sentences.items():
            cluster_indices = np.where([cluster_assignment[i] == cluster_id for i in range(len(cluster_assignment))])[0]
            cluster_embeddings = corpus_embeddings[cluster_indices]
            cluster_centroid = np.mean(cluster_embeddings, axis=0)
            distances = np.linalg.norm(cluster_embeddings - cluster_centroid, axis=1)
            key_sentence_index = np.argmin(distances)
            key_sentences[cluster_id] = clean_text[cluster_indices[key_sentence_index]]

        # Count the number of sentences in each cluster
        cluster_sentence_counts = {cluster_id: len(sentences) for cluster_id, sentences in clustered_sentences.items()}
        
        data = []
        for cluster_id, sentences in clustered_sentences.items():
            
            for sentence in sentences:
                data.append({
                    "Cluster": cluster_id + 1,
                    "Question": sentence,
                    "Sentence Count": cluster_sentence_counts[cluster_id],
                    "Topic": sentence == key_sentences[cluster_id]
                })
                
        df = pd.DataFrame(data)
        self.update_google_sheet(df.sort_values(by=['Cluster']))
        logging.info("DataFrame prepared for Google Sheet update.")


if __name__ == '__main__':
    pipeline = TextClusteringPipeline(SPREADSHEET_ID, SHEET_NAME, PROJECT_ID, DATASET_ID, TABLE_ID)
    pipeline.run()

2024-06-18 18:08:20,559 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-06-18 18:08:20,806 - INFO - Use pytorch device_name: cpu
2024-06-18 18:08:20,807 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L12-v2
2024-06-18 18:08:23,152 - INFO - Pipeline started.
2024-06-18 18:08:26,310 - INFO - Data successfully retrieved from BigQuery.


Batches:   0%|          | 0/123 [00:00<?, ?it/s]

2024-06-18 18:08:47,988 - INFO - Embeddings generated.


      Cluster                                           Question  \
0          36                                      Good mormning   
1          36  Good start, but this is not what I was looking...   
2          36                                I think it's wrong.   
3          36                       this is not the correct one.   
4          36                          that’s incorrect feedback   
...       ...                                                ...   
3918      389                                     explain E-Line   
3919      389      under what architecture does the e-line work?   
3920      275          how do i turn up new routers step by step   
3921      275  how do i turn up new routers step by step for ...   
3922      275  can you please provide a step by step guide on...   

      Sentence Count  Topic  
0                  7  False  
1                  7  False  
2                  7  False  
3                  7  False  
4                  7  False  
...

2024-06-18 18:08:52,681 - INFO - Header updated in Google Sheet.
2024-06-18 18:08:54,588 - INFO - 15692 cells updated in Google Sheet.
2024-06-18 18:08:54,590 - INFO - DataFrame prepared for Google Sheet update.


### Test